In [207]:
import numpy as np 
import pandas as pd 
import copy
from rnanorm import TMM
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
import scipy.stats as stats
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import RobustScaler
from scipy.stats import pearsonr

import networkx as nx
import community

/tmp/ipykernel_2560/428407731.py:9: DeprecationWarning: Please import `pearsonr` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.stats.stats import pearsonr


In [208]:
import os

In [209]:
expr_path=os.listdir('../Data/GSE162694/gene counts/')

In [210]:
traits_path=os.listdir('../Data/GSE162694/data traits/')

In [238]:
data_expr_frame=[]
for i in range(-1,len(expr_path)):
    data_expr_stage=pd.read_csv(f'../Data/GSE162694/gene counts/GSE162694_stage_{i}.csv')
    data_expr_frame.append(data_expr_stage)
data_expr=pd.concat(data_expr_frame)

FileNotFoundError: [Errno 2] No such file or directory: '../Data/GSE162694/gene counts/GSE162694_stage_5.csv'

In [ ]:
data_expr.rename(columns={'Source Name':'Sample Name'},inplace=True)


In [221]:
data_expr=data_expr.set_index('Sample Name')

mean=data_expr.T.mean()

In [241]:
q=0.5
threshold=np.quantile(mean,q)
while threshold>500 and q>=0.05:
    q=q-0.05
    print(q)
    threshold=np.quantile(mean,q)
data_expr.sort_index(inplace=True)
data_expr=data_expr.T
data_expr.sort_index(inplace=True)
data_expr=data_expr.loc[data_expr.mean(axis=1)>threshold]

0.45
0.4
0.35000000000000003
0.30000000000000004
0.25000000000000006
0.20000000000000007
0.15000000000000008
0.10000000000000007
0.05000000000000007
6.938893903907228e-17
-0.04999999999999993


ValueError: Quantiles must be in the range [0, 1]

In [199]:
threshold

57476.90997067449

In [ ]:

norm_expr=TMM().fit_transform(data_expr)
gene_counts_normalized=pd.DataFrame(norm_expr)
gene_counts_normalized.columns=data_expr.columns
gene_counts_normalized=gene_counts_normalized.T
gene_counts_normalized.columns=data_expr.T.columns

scaler = RobustScaler()

scaler.fit(gene_counts_normalized)

gene_counts_normalized = scaler.transform(gene_counts_normalized)

gene_counts_normalized=pd.DataFrame(gene_counts_normalized)
gene_counts_normalized.columns=data_expr.T.columns
gene_counts_normalized=gene_counts_normalized.T
gene_counts_normalized.columns=data_expr.columns


pearson_corr_df=gene_counts_normalized.T.corr(method='pearson')
pearson_corr_df = pearson_corr_df.stack().reset_index()
pearson_corr_df.columns = ['gene1', 'gene2', 'pearson_corr']
pearson_corr_df['weight']=abs((1+pearson_corr_df['pearson_corr'].values)/2)**2

filtered_corr_df=pearson_corr_df.loc[(pearson_corr_df['pearson_corr']>=0.65)&(pearson_corr_df['pearson_corr']<1)]
print(len(filtered_corr_df))
G=nx.Graph()
for i in range(len(filtered_corr_df)):
    G.add_edge(filtered_corr_df.gene1.values[i], filtered_corr_df.gene2.values[i], \
            weight=filtered_corr_df.weight.values[i])


In [175]:
data_traits_frame=[]
for i in range(-1,len(traits_path)-1):
    data_traits_stage=pd.read_csv(f'../Data/GSE162694/data traits/GSE162694_datatraits_stage_{i}.csv')
    data_traits_frame.append(data_traits_stage)
data_traits=pd.concat(data_traits_frame)

In [176]:
# rem=data_traits.loc[data_traits['Factor Value[disease]']=='alcoholic liver disease']['Source Name'].values

In [177]:
# data_traits=data_traits.loc[~(data_traits['Factor Value[disease]']=='alcoholic liver disease')]

In [ ]:
def kmeans_pcc_pca(X, n_clusters, init_centroids, max_iters=15):
    """
    K-means clustering using PCC (Pearson Correlation Coefficient) as distance metric
    and PCA (Principal Component Analysis) for center clustering.
    
    Args:
        X (np.ndarray): Data matrix (each row represents a gene).
        n_clusters (int): Number of clusters.
        max_iters (int): Maximum number of iterations.
    
    Returns:
        np.ndarray: Cluster assignments for each gene.
    """
    _,n_genes = X.shape
    centroids = init_centroids
    prev_assignments = np.zeros(n_genes)
    for k in range(max_iters):
        # Calculate pairwise PCC distances
        print(k)
        distances=[]
        for i in range(n_clusters):
            distance=[]
            for j in range(n_genes):
                d=1-abs(((1+(pearsonr(X.iloc[:,j].values,centroids[i])[0]))/2))**2
                distance.append(d)
            distances.append(distance)

        # Assign each gene to the nearest centroid
        assignments = np.argmin(distances, axis=0)
        temp=0
        # Update centroids using PCA
        new_centroids=[]
        for i in range(n_clusters):
            cluster_genes = X.T[assignments == i]
            if cluster_genes.shape[0]==0:
                temp+=1
                continue
            pca = PCA(n_components=1)
            centroid_pca = pca.fit_transform(cluster_genes.T)
            new_centroids.append(centroid_pca.T[0])
        centroids=new_centroids
        n_clusters-=temp

        # Step 9: Check convergence

        if np.array_equal(assignments, prev_assignments):
            break
        prev_assignments = assignments
    
    return assignments

In [ ]:
def preprocess(data_expr_frame):
    data_expr=data_expr_frame.set_index('Sample Name')
    data_expr=data_expr.T
    data_expr.drop(['Sample 48', 'Sample 58'],inplace=True, axis=1)
    data_expr=data_expr.T
    mean=data_expr.T.mean()
    q=0.5
    threshold=np.quantile(mean,q)
    if threshold>500:
        q+=0.05
        threshold=np.quantile(mean,q)
    data_expr.sort_index(inplace=True)
    data_expr=data_expr.T
    data_expr.sort_index(inplace=True)
    data_expr=data_expr.loc[data_expr.mean(axis=1)>threshold]
    norm_expr=TMM().fit_transform(data_expr)
    gene_counts_normalized=pd.DataFrame(norm_expr)
    gene_counts_normalized.columns=data_expr.columns
    gene_counts_normalized=gene_counts_normalized.T
    gene_counts_normalized.columns=data_expr.T.columns

    scaler = RobustScaler()

    scaler.fit(gene_counts_normalized)

    gene_counts_normalized = scaler.transform(gene_counts_normalized)

    gene_counts_normalized=pd.DataFrame(gene_counts_normalized)
    gene_counts_normalized.columns=data_expr.T.columns
    gene_counts_normalized=gene_counts_normalized.T
    gene_counts_normalized.columns=data_expr.columns


    pearson_corr_df=gene_counts_normalized.T.corr(method='pearson')
    pearson_corr_df = pearson_corr_df.stack().reset_index()
    pearson_corr_df.columns = ['gene1', 'gene2', 'pearson_corr']
    pearson_corr_df['weight']=abs((1+pearson_corr_df['pearson_corr'].values)/2)**2

    filtered_corr_df=pearson_corr_df.loc[(pearson_corr_df['pearson_corr']>=0.65)&(pearson_corr_df['pearson_corr']<1)]
    print(len(filtered_corr_df))
    G=nx.Graph()
    for i in range(len(filtered_corr_df)):
        G.add_edge(filtered_corr_df.gene1.values[i], filtered_corr_df.gene2.values[i], \
                weight=filtered_corr_df.weight.values[i])

    louvain_clusters = nx.community.louvain_communities(G, weight='weight',resolution=5)

    clusters = {node: community_id for community_id, community \
                in enumerate(louvain_clusters) for node in community}
    louvain_clusters_df=pd.DataFrame({'gene':clusters.keys(),
                                    'cluster':clusters.values()})
    temp=pd.DataFrame(louvain_clusters_df.cluster.value_counts())
    temp=temp.loc[temp['count']>50]
    selected_cluster=temp.index
    louvain_clusters_df=louvain_clusters_df.loc[louvain_clusters_df['cluster'].isin(selected_cluster)]
    selected_gene_counts_normalized=gene_counts_normalized.T[louvain_clusters_df.gene]
    selected_gene_counts_normalized=selected_gene_counts_normalized.T.sort_index().T

    centroids=[]
    for i in louvain_clusters_df.cluster.unique():
        pca=PCA(n_components=1)
        X=gene_counts_normalized.T[louvain_clusters_df.loc[louvain_clusters_df['cluster']==i]['gene']]
        X_pca=pca.fit_transform(X)
        centroids.append(X_pca.T[0])

            # Set the number of clusters
    n_clusters = len(louvain_clusters_df.cluster.unique())

    # Run K-means with PCC distance and PCA-based center clustering
    cluster_assignments = kmeans_pcc_pca(selected_gene_counts_normalized, n_clusters,centroids)
    cluster_assignments_df=pd.DataFrame(cluster_assignments,columns=['cluster'])
    
    selected_gene_counts_normalized=selected_gene_counts_normalized.T

    selected_gene_counts_normalized['cluster']=cluster_assignments_df.cluster.values
    t=cluster_assignments_df['cluster'].value_counts()
    rem=[]
    for i in range(len(t.keys())):
        if t.values[i]<50:
            rem.append(t.keys()[i])
    selected_gene_counts_normalized=selected_gene_counts_normalized.loc \
        [~selected_gene_counts_normalized['cluster'].isin(rem)]

    return selected_gene_counts_normalized

In [ ]:
# pval=[]
# for i in range(len(filtered_2_corr_df)):
#     pval.append(pearsonr(df.T[filtered_2_corr_df.iloc[i]['gene1']],df.T[filtered_2_corr_df.iloc[i]['gene2']])[1])
# filtered_2_corr_df['pval']=pval
# filtered_2_corr_df

In [ ]:
cluster_assignments_df=preprocess(data_expr)

In [ ]:
cluster_assignments_df.cluster.value_counts()

In [ ]:
t=cluster_assignments_df

In [ ]:
t

In [ ]:
cluster_assignments_df.to_csv('../Result/E-MTAB-6863/data_expr.csv')

In [ ]:
# selected_genes_frame=pd.read_csv('../Result/GSE135251/gene_expr_12.csv').rename(columns={'Unnamed: 0':'Sample Name'}).set_index('Sample Name')
# selected_genes_frame

In [ ]:
def corr_normalized(selected_gene_counts_normalized,i):
    gene_counts_normalized=selected_gene_counts_normalized.loc[selected_gene_counts_normalized['cluster']==i].drop('cluster',axis=1)
    return gene_counts_normalized

In [ ]:
def calculateME(df):
    pca=PCA(n_components=1)
    X=pca.fit_transform(df.T).flatten()
    return X

In [ ]:
def evaluateModule(df1,df2):
    score=0
    for i in range(len(df1)):
        me_i=calculateME(df1)
        corr_i=pearsonr(df1.iloc[i,:].values,me_i)[0]
        for i in range(len(df2.cluster.unique())):
            cluster_df2=df2.loc[df2['cluster']==i].drop('cluster',axis=1)
            me_j=calculateME(cluster_df2)
            corr_j=pearsonr(df1.iloc[i,:],me_j)[0]

            if corr_i>=corr_j:
                score+=1
    return float(score/(len(df1)*len(df2.cluster.unique())))

In [ ]:
score=[]
for i in range(len(cluster_assignments_df['cluster'].unique())):
    module=cluster_assignments_df.loc[cluster_assignments_df['cluster']==i].drop('cluster',axis=1)
    score.append(evaluateModule(module,cluster_assignments_df))

In [ ]:
pd.DataFrame(score).to_csv('../Result/E-MTAB-6863/evaluation_score.csv')

In [ ]:
fs=data_traits.reset_index(drop=True)['fibrosis stage']
# .apply(lambda x:0 if x<4 else 1)
dummies_stage_df=pd.get_dummies(fs, dtype=float)

In [ ]:
module_corrs=[]
index=[]
for i in cluster_assignments_df.cluster.unique():  
    pearson_corrs=[]
    print(f"cluster {i}")
    for j in range(5):
        # Áp dụng PCA lên dữ liệu
        pca=PCA(n_components=1)
        module=pca.fit_transform(corr_normalized(cluster_assignments_df,i).T).T[0]
        rho, p = pearsonr(module, dummies_stage_df.iloc[:,j].values)
        print(rho)
        pearson_corrs.append(rho)
    index.append(i)
    module_corrs.append(pearson_corrs)

In [ ]:
max(np.array(module_corrs).flatten())

In [ ]:
cluster_assignments_df=cluster_assignments_df.T

In [ ]:
import seaborn as sns

In [ ]:
heat_data=pd.DataFrame(module_corrs).reset_index(drop=True)
heat_data['module']=np.array(index)+1
heat_data=heat_data.set_index('module')
heat_data.sort_index(inplace=True)
fsc_png=sns.heatmap(heat_data, annot=True,vmin=-1, vmax=1).get_figure()
plt.xlabel('fibrosis stage', fontsize = 10) 
plt.show()

In [ ]:
fsc_png.savefig('../Result/E-MTAB-6863/module-traits.png')

In [ ]:
# !pip install gprofiler-official

In [ ]:
# !pip install sanbomics

In [ ]:
from sanbomics.tools import id_map
from gprofiler import GProfiler

In [ ]:
# cluster_assignments_df=pd.read_csv('../Result/GSE135251/data_expr.csv')

In [ ]:
cluster_assignments_df=cluster_assignments_df.T

In [ ]:
import math

In [ ]:
# sigi=[]
# # Set the gene symbols for the module
# for i in cluster_assignments_df['cluster'].unique():
#     module_genes = list(cluster_assignments_df.loc[cluster_assignments_df['cluster']==i].index)

#     # Initialize the g:Profiler object
#     gp = GProfiler(return_dataframe=True)

#     # Perform functional enrichment analysis
#     enrichment_results = gp.profile(organism='hsapiens',
#                                     query=module_genes,
#                                     no_evidences=False,
#                                     ordered=True)

#     # View the enrichment results
#     df=pd.DataFrame(enrichment_results)
#     df['sig']=-df['p_value'].apply(lambda x: math.log10(x))
#     sigj=df['sig'].sum()
#     sigi.append(sigj)

In [ ]:
# sum(sigi)/len(cluster_assignments_df['cluster'].unique())

In [ ]:
# sum(sigi)/len(cluster_assignments_df['cluster'].unique())/len(cluster_assignments_df)

In [ ]:
module_genes = list(cluster_assignments_df.loc[cluster_assignments_df['cluster']==4].index)

# Initialize the g:Profiler object
gp = GProfiler(return_dataframe=True)

# Perform functional enrichment analysis
enrichment_results = gp.profile(organism='hsapiens',
                                query=module_genes,
                                no_evidences=False,
                                ordered=True)

# View the enrichment results
df=pd.DataFrame(enrichment_results)
df.sort_values('p_value',ascending=True,inplace=True)

In [ ]:
df=df[['source','native','name','p_value','description']]

In [ ]:
df.head(5).to_csv('e-mtab-6863-module5-stage0.csv')

In [ ]:
t=cluster_assignments_df.loc[cluster_assignments_df['cluster']==3].drop('cluster',axis=1)

In [ ]:
key_module_corr=t.T.corr(method='pearson')

In [ ]:
key_module_corr = key_module_corr.stack().reset_index()
key_module_corr.columns = ['gene1', 'gene2', 'pearson_corr']

key_module_corr.pearson_corr=key_module_corr.pearson_corr.apply(abs)

In [ ]:
hub_gene=key_module_corr[['gene1','pearson_corr']].groupby('gene1').sum() \
    .sort_values('pearson_corr',ascending=False).reset_index().head(10)

In [ ]:
hub_gene